# Modular Arithmetic

---

In [1]:
from time import perf_counter # Don't use time.time to check performance!

## Calculating $a^b \space mod \space m$

[Reference](https://stackoverflow.com/questions/2177781/how-to-calculate-modulus-of-large-numbers)

Let's calculate $5^{55} \space mod \space 221$, for example. At first we'll take a naïve approach, and then refine it.

The idea is to calculate $a_1=a \times a_0 \space (mod \space m)$, whence it follows that $0 \le a_1 < m$. Then, we'll repeatedly apply $a_{n+1}=a \times a_n \space (mod \space m)$ until we reach $a_n=a_b$. Note that $a_0 \equiv 1$.

Using this method, we avoid encountering numbers larger than $m^2$ because at every step $0 \le a_n < m$ and $0 \le a_1 < m$.

In [9]:
(5**55) % 221

112

In [3]:
def simple_reduce_exponent(a: int, b: int, m: int):
    res = 1
    while b > 0:
        res = (res * a) % m
        b -= 1
    return res

In [4]:
simple_reduce_exponent(5, 55, 221)

112

We've verified the scheme works; this is useful when we're dealing with really large numbers that can't be calculated directly:

In [5]:
simple_reduce_exponent(25519, 42048501, 221)

117

We can improve our method by using [exponentiation by squaring](https://en.wikipedia.org/wiki/Exponentiation_by_squaring); hence we will reduce our function to use only $log \space b$ multiplications instead of $b$. This is the [right-to-left binary method](https://en.wikipedia.org/wiki/Modular_exponentiation#Right-to-left_binary_method).

In [6]:
def reduce_exponent(a: int, b: int, m: int):
    a = a % m
    res = 1
    while b > 0:
        if b % 2 == 1:
            res = (res * a) % m
        b = b // 2
        a = (a * a) % m
    return res

In [7]:
reduce_exponent(5, 55, 221)

112

Let's quantify our improvement with `time.perf_counter`:

In [8]:
start = perf_counter()
simple_reduce_exponent(25519, 42048501, 221)
end = perf_counter()
t1 = end - start

start = perf_counter()
reduce_exponent(25519, 42048501, 221)
end = perf_counter()
t2 = end - start

{'simple': t1, 'improved': t2, 'diff': t1 - t2}

{'simple': 5.813194399990607,
 'improved': 5.1499984692782164e-05,
 'diff': 5.813142900005914}